In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sys
import os
import glob
import scipy.stats as sps
from tqdm import tqdm

In [2]:
def rank_zscore(data):
    """ 
    Return the normally-distributed z-scores of the given data. Each column is individually 
    ranked against itself, then the ranked data is z-scored assuming a normal distribution.
    
    
    Parameters
    ----------
    data : (G, S) array
        Array with ``G`` genes and ``S`` independent samples containing scRNA-seq data.
    
    Returns
    -------
    out : (G, S) array
        Array with ``G`` genes and ``S`` independent samples containing ranked then z-scored scRNA-seq data.
    
    """

    if len(data.shape) == 1:
        data = data.reshape((data.size,1))
        
    G, S = data.shape
    
    # Calculate rank for each sample
    rank_data = np.zeros(data.shape) 
    
    for i in range(S):
        # by default, ties method is average
        rank_data[:,i] = sps.rankdata(data[:,i]) 

    # Convert rank_data to percentiles according to a normal distribution
    output_data = sps.norm.ppf(rank_data / (G+1))

    return output_data

def process(data, average=False):
    """
    Return the processed scRNA-seq data. Each sample is independently normalized, then the fitted z-score
    is calculated. Optionally, an average is taken across samples before z-scoring.
    
    Parameters
    ----------
    data : (G, S) DataFrame
        Table with ``G`` genes and ``S`` independent samples containing raw counts from scRNA-seq.
    average : bool, optional
        if True: take the average across samples after normalizing and before z-scoring
        
    Returns
    -------
    out : (G, S) DataFrame (or (G, 1) if ``average``)
        Table with ``G`` genes and ``S`` independent samples containing processed scRNA-seq data.
    
    """
    
    # Normalize each sample independently
    data_normalized = data/np.sum(data,axis=0)
    
    # Average across samples, if requested
    if average:
        data_normalized = data_normalized.mean(axis=1)
    
    # Find the normal-distribution z-scores of log(expression + 1)
    data_zscored = rank_zscore(np.log2(data_normalized.values + 1))
    
    return pd.DataFrame(data_zscored, index=data.index)

In [3]:
source_folder = './source_data/'
dest_folder = './processed_data/'

# Load [Mouse Cell Atlas](https://figshare.com/articles/dataset/MCA_DGE_Data/5435866)

840 cell types

In [4]:
MCA_folder = source_folder + 'MCA_rmbatch_dge/'
MCA_annotations = pd.read_csv(MCA_folder + 'MCA_CellAssignments.csv')
MCA_annotations = MCA_annotations.drop(columns='Unnamed: 0')

cell_types = MCA_annotations['Annotation'].unique()
dataframes_specific = {cell_type: [] for cell_type in cell_types}

Make a dictionary of cell type:dataframes. Each dataframe corresponds to all samples of a particular cell type.

**This cell takes hours to run. Avoid re-running if possible!**

In [ ]:
unlabeled_cells = []

for filename in tqdm(glob.glob(MCA_folder + '*_rm.batch_dge.txt'), desc='File progress'):
    current_dataframe = pd.read_csv(filename, sep=" ")
    
    for current_cell in current_dataframe.columns:
        if len(np.where(MCA_annotations['Cell.name'] == current_cell)[0]) == 0:
            unlabeled_cells += [current_dataframe[current_cell]]
        else:
            current_type = MCA_annotations['Annotation'][MCA_annotations['Cell.name'] == current_cell]
            dataframes_specific_type[current_type.values[0]] += [current_dataframe[current_cell]]
            
# Save the dictionary of dataframes, since it takes 3 hours (!) to create
f = open(dest_folder+'MCA_dictionary.dat','wb')
pickle.dump(dataframes_specific_type,f)
f.close()

# Save the dataframes to separate .h5 files, too
for cell_type, dataframe in tqdm(dataframes_specific_type.items(), desc='Save progress'):
    type_string = cell_type.replace('/','-')
    dataframe.to_hdf(dest_folder+'MCA_raw_2022/'+type_string + '.h5', key = 'df', mode = 'w')

In [ ]:
import pickle

f = open('/projectnb/biophys/mariay/MCA/MCA_dictionary.dat','wb')
pickle.dump(dataframes_specific_type,f)
f.close()

In [ ]:
dataframes_specific = {cell_type: pd.DataFrame() for cell_type in cell_types}

for cell_type, array in tqdm(dataframes_specific_type.items()):
    dataframes_specific[cell_type] = pd.concat(array, axis='columns')
    
    type_string = cell_type.replace('/','-')
    dataframes_specific[cell_type].to_hdf(dest_folder+'MCA_raw_2022/'+type_string + '.h5', key = 'df', mode = 'w')

Average and z-score the samples corresponding to a particular cell type, then concatenate each processed single-column dataframe into one big dataframe that will form the basis.

In [5]:
# Standardize label format
tags = pd.DataFrame([], columns=['Organ', 'Cell Type', 'Time Point', 'Condition', 'Source', 'Cell Count'], index=cell_types)

for label in cell_types:
    # Remove all _ and -
    cleaned_label = label

    if 'Pregrancy' in cleaned_label:
        cleaned_label = cleaned_label.replace('Pregrancy','Pregnancy')
    if 'Stomache' in cleaned_label:
        cleaned_label = cleaned_label.replace('Stomache','Stomach')
    if 'β' in cleaned_label:
        cleaned_label = cleaned_label.replace('β','Beta')
    if 'Clara' in cleaned_label:
        cleaned_label = cleaned_label.replace('Clara','Club')
    if 'Preosteoblast-Osteoblast-Bone cell-Cartilage' in cleaned_label:
        cleaned_label = cleaned_label.replace('Preosteoblast-Osteoblast-Bone cell-Cartilage',
                                              'Preosteoblast/Osteoblast/Bone cell/Cartilage')
    if 'Tendon stem-progenitor' in cleaned_label:
        cleaned_label = cleaned_label.replace('Tendon stem-progenitor',
                                              'Tendon stem/progenitor')
        
    if 'Neonatal' in cleaned_label:
        tags['Time Point'][label] = 'P1'
        cleaned_label = cleaned_label.replace('Neonatal','')
    elif 'Fetal' in cleaned_label:
        tags['Time Point'][label] = 'E14.5'
        cleaned_label = cleaned_label.replace('Fetal','')
    else:
        tags['Time Point'][label] = 'WK6-10'
        
    if 'Lactation' in cleaned_label:
        tags['Condition'][label] = 'lactation'
        cleaned_label = cleaned_label.replace('Lactation','')
    if 'Involution' in cleaned_label:
        tags['Condition'][label] = 'involution'
        cleaned_label = cleaned_label.replace('Involution','')
    if 'Virgin' in cleaned_label:
        tags['Condition'][label] = 'virgin'
        cleaned_label = cleaned_label.replace('Virgin','')
    if 'high' in cleaned_label:
        underscore_idx = cleaned_label.find('_')
        high_idx = cleaned_label.find('high')
        high_gene = cleaned_label[underscore_idx+1:high_idx]
        tags['Condition'][label] = high_gene + ' high'
        cleaned_label = cleaned_label.replace('_' + high_gene + 'high', '')
        
    cleaned_label = cleaned_label.replace('_',' ')
    cleaned_label = cleaned_label.replace('-',' ')
    
    paren_idx1 = cleaned_label.find('(')
    paren_idx2 = cleaned_label.find(')')
    
    # Make sure the first letter of each word is capitalized
    cleaned_label = ' '.join(word[0].upper() + word[1:] for word in cleaned_label.split())
    
    if label=='Astroglial Cell (Bergman Glia) (Brain)': # this is the only type with 2 sets of parentheses
        tags['Organ'][label] = 'Brain'
        tags['Cell Type'][label] = 'Bergmann Glia'
    else:
        tags['Organ'][label] = cleaned_label[paren_idx1+1:paren_idx2].strip()
        tags['Cell Type'][label] = cleaned_label[:paren_idx1].strip()
    
    tags['Source'][label] = 'MC20'
    
tags = tags.fillna('')

In [6]:
general_types_map = tags['Organ'] + ' ' + tags['Cell Type'] + ' ' + tags['Time Point'] + ' ' + tags['Source']
general_types = general_types_map.unique()
dataframes_general = {cell_type:[] for cell_type in general_types}

In [7]:
# Retrieve raw sample data that has been separated into individual files/dataframes according to cell type
# Make each entry in the dataframes_general_type dictionary an array of dataframes, that we will concatenate

for file in tqdm(glob.glob("./processed_data/MCA_raw/*.h5")):
    current_df = pd.read_hdf(file)
    current_type = file[len('./processed_data/MCA_raw/'):-3]
    
    if 'Preosteoblast-Osteoblast-Bone cell-Cartilage' in current_type:
        current_type = current_type.replace('Preosteoblast-Osteoblast-Bone cell-Cartilage',
                                              'Preosteoblast/Osteoblast/Bone cell/Cartilage')
    if 'Tendon stem-progenitor' in current_type:
        current_type = current_type.replace('Tendon stem-progenitor',
                                              'Tendon stem/progenitor')
    dataframes_general[general_types_map[current_type]] += [current_df]

100%|██████████| 840/840 [04:27<00:00,  3.14it/s]


In [8]:
all_genes = np.array([])

for cell_type, array in tqdm(dataframes_general.items()):
    # Concatenate all samples belonging to a general type
    dataframes_general[cell_type] = pd.concat(array, axis='columns')
    
    # Collect all genes so we can fill missing entries with zeroes
    all_genes = np.unique([*all_genes, *dataframes_general[cell_type].index])

100%|██████████| 516/516 [00:26<00:00, 19.78it/s]


In [9]:
for cell_type, dataframe in tqdm(dataframes_general.items()):
    # Fill genes with no entry with zeroes
    dataframes_general[cell_type] = pd.DataFrame(dataframe,index=all_genes).fillna(0)

100%|██████████| 516/516 [02:51<00:00,  3.00it/s]


In [10]:
tags_general = pd.DataFrame([], columns=['Organ', 'Cell Type', 'Time Point', 'Condition', 'Source', 'Cell Count'],
                            index=general_types)

for label in general_types:
    # Use tags info from one of the corresponding specific types (everything except condition and cell count is the same)
    specific_type = general_types_map[general_types_map == label].index[0]
    tags_general.loc[label] = tags.loc[specific_type]
    
tags_general = tags_general.drop(columns='Condition')

In [11]:
dataframe_list = []

for cell_type, dataframe in tqdm(dataframes_general.items(), desc='Type progress'):
    tags_general.loc[cell_type, 'Cell Count'] = dataframe.shape[1]
    processed_dataframe = process(dataframe, average=True)
    processed_dataframe.index.name = 'Gene'
    processed_dataframe.columns = [cell_type]
    dataframe_list += [processed_dataframe]
    
mca_data = pd.concat(dataframe_list, axis='columns')

Type progress: 100%|██████████| 516/516 [02:10<00:00,  3.95it/s]


In [27]:
mca_data.to_hdf(dest_folder + 'MCA_basis_apr22.h5', key = 'df', mode = 'w')
tags_general.to_csv(dest_folder + 'MCA_basis_apr22_metadata.csv')

Replace some MCA lung samples with Michael Herriges' control samples

In [12]:
combined_file = pd.read_csv(source_folder + 'herriges/21_01_06_herriges_with_week6_invivo.csv')
combined_metadata = pd.read_csv(source_folder + 'herriges/metadata.csv')

# Michael's labels for each of the clusters
cluster_key = {'2':'AT1 and AT1-like',
               '4':'Ciliated',
               '6':'Basal',
               '7':'Gastric-like',
               '8':'Neuroendocrine',
               '0+':'AT2',
               '1+13':'Secretory',
               '3+14':'AT2-like'}

clusters = combined_metadata['new_clustering.07.12.2021'].values
cluster_labels = [cluster_key[cluster] for cluster in clusters]
identity = combined_metadata['orig.ident'].values

def extract_sample(cell_type, cell_source):
    indices = np.intersect1d(np.where(np.array(cluster_labels)==cell_type)[0],np.where(np.array(identity)==cell_source)[0])

    if len(indices)==0:
        raise LookupError('No cells in the specified subset')
        
    sample = process(combined_file.iloc[:,indices], average=True)
    
    return sample, len(indices)

In [13]:
mca_herriges_data = mca_data.copy()
tags_herriges = tags_general.copy()

herriges_types = {'Lung AT1 Cell WK6-10 MC20':'AT1 and AT1-like',
                  'Lung AT2 Cell WK6-10 MC20':'AT2',
                  'Lung Ciliated Cell WK6-10 MC20':'Ciliated',
                  'Lung Club Cell WK6-10 MC20':'Secretory'
                 }

herriges_list = []

for cell_type, cluster in herriges_types.items():
    new_tag = cell_type.replace('MC20', 'KO22')
    new_tag = new_tag.replace('-10', '')
    mca_herriges_data = mca_herriges_data.drop(columns=cell_type)
    sample_data, sample_cell_count = extract_sample(cluster, 'Control_wk6')
    mca_herriges_data = pd.concat([mca_herriges_data, 
                                   pd.DataFrame(sample_data, columns=[new_tag])], axis='columns')
    
    tags_herriges.loc[new_tag] = tags_herriges.loc[cell_type]
    tags_herriges = tags_herriges.drop(index=cell_type)
    tags_herriges['Source'][new_tag] = 'KO22'
    tags_herriges['Time Point'][new_tag] = 'WK6'
    tags_herriges['Cell Count'][new_tag] = sample_cell_count

basal_tag = 'Lung Basal Cell WK6 KO22'
sample_data, sample_cell_count = extract_sample('Basal', 'Control_wk6')
mca_herriges_data = pd.concat([mca_herriges_data, 
                               pd.DataFrame(sample_data, columns=[basal_tag])], axis='columns')
    
tags_herriges.loc[basal_tag] = {'Organ':'Lung',
                                'Cell Type':'Basal',
                                'Time Point': 'WK6',
                                'Source':'KO22',
                                'Cell Count': sample_cell_count}

In [68]:
[sample for sample in mca_herriges_data.columns if 'Lung' in sample]

['Lung Stromal Cell E14.5 MC20',
 'Lung Erythroblast E14.5 MC20',
 'Lung Neutrophil E14.5 MC20',
 'Lung Smooth Muscle Cell E14.5 MC20',
 'Lung Macrophage E14.5 MC20',
 'Lung Epithelial Cell E14.5 MC20',
 'Lung Dendritic Cell WK6-10 MC20',
 'Lung Ig−producing B Cell WK6-10 MC20',
 'Lung Conventional Dendritic Cell WK6-10 MC20',
 'Lung Endothelial Cells WK6-10 MC20',
 'Lung Stromal Cell WK6-10 MC20',
 'Lung T Cell WK6-10 MC20',
 'Lung B Cell WK6-10 MC20',
 'Lung Nuocyte WK6-10 MC20',
 'Lung Dividing Dendritic Cells WK6-10 MC20',
 'Lung Alveolar Macrophage WK6-10 MC20',
 'Lung Interstitial Macrophage WK6-10 MC20',
 'Lung Plasmacytoid Dendritic Cell WK6-10 MC20',
 'Lung Monocyte Progenitor Cell WK6-10 MC20',
 'Lung Endothelial Cell WK6-10 MC20',
 'Lung Neutrophil Granulocyte WK6-10 MC20',
 'Lung NK Cell WK6-10 MC20',
 'Lung Eosinophil Granulocyte WK6-10 MC20',
 'Lung Basophil WK6-10 MC20',
 'Lung Dividing T Cells WK6-10 MC20',
 'Lung Dividing Cells WK6-10 MC20',
 'Lung Alveolar Bipotent Pr

In [14]:
mca_herriges_data.loc[:,'Lung AT1 Cell WK6 KO22']

00R_AC107638.2   NaN
0610005C13Rik    NaN
0610006L08Rik    NaN
0610007P14Rik    NaN
0610009B22Rik    NaN
                  ..
Gm30893          NaN
Gm47118          NaN
Gm34558          NaN
Gm36377          NaN
AC138604.2       NaN
Name: Lung AT1 Cell WK6 KO22, Length: 38353, dtype: float64